In [3]:
import os, sys
from pathlib import Path

def find_src_folder():
    current = Path(os.getcwd()).resolve()
    for p in [current] + list(current.parents):
        src = p / "src"
        if src.exists():
            return src
    raise RuntimeError("src 폴더를 찾을 수 없습니다.")

src_path = find_src_folder()
sys.path.append(str(src_path))

In [67]:
system_prompt = """
너는 한국어 STT 결과가 "인지 가능"인지 "인지 불가"인지
이진 분류용 학습데이터를 생성하는 역할이다.

데이터는 (stt_text, label) 쌍으로만 구성된다.

판정 기준은 오직 하나다.
- stt_text만 단독으로 보았을 때,
  사람이 어떤 질문을 하려는지 감이 오면 label=0
- 감이 오지 않으면 label=1

문법, 맞춤법, 조사, 띄어쓰기, 단어 치환은 전혀 중요하지 않다.
틀려도 된다.
짧아도 된다.

중요한 것은 오직 이것이다:
→ "사람이 의미를 떠올릴 수 있느냐"

아래와 같은 형태는 반드시 label=0이다.
- "이벤터 어또케"
- "몬스터 모야"
- "인벤 뭐있어"
- "보스방 어케가"

출력은 JSONL 형식만 허용한다.
설명, 주석, 추가 텍스트는 출력하지 않는다.
"""

human_prompt_0 = """
이번에 생성하는 모든 샘플의 label은 반드시 0이다.

아래 조건을 만족하는 한국어 stt_text를 생성하라.

조건:
- STT 인식 결과처럼 보이는 문장
- 발음이 찌그러지거나 단어가 틀려도 상관없다
- 조사 누락, 축약, 띄어쓰기 붕괴 모두 허용
- 하지만 사람이 보면
  "아, 이게 무슨 질문인지 알겠다"는 느낌이 들어야 한다
- 시스템 프롬프트를 참고해서 세계관에 맞는 stt_text를 위주로 만들어달라

의도가 느껴지면 된다.
완벽할 필요는 없다.

출력은 JSONL 한 줄만 한다.

{
  "stt_text": "...",
  "label": 0
}
"""

human_prompt_1 = """
이번에 생성하는 모든 샘플의 label은 반드시 1이다.

중요:
label 판단은 오직 stt_text만 보고 한다.
비교 대상은 없다.
정답 문장을 상상하지 마라.

아래 조건을 모두 만족하는 stt_text만 생성하라.

조건:
- 사람이 봤을 때 어떤 질문인지 감이 오지 않아야 한다
- 핵심 명사나 동작이 드러나면 실패다
- 문장이 질문처럼 느껴지면 실패다
- "이게 무슨 말이지?"라는 반응이 나와야 한다
- 가끔 영어랑 한글이 섞여있다.
- 한자도 섞일때가 있다.

다음과 같은 형태는 절대 만들지 마라 (이건 label=0이다):
- 인벤 아이템 무슨키지
- 이벤트 어떻게 해
- 몬스터 뭐야
- 보스방 가는길

반드시 더 심하게 깨뜨려라.

출력은 JSONL 한 줄만 한다.

{
  "stt_text": "...",
  "label": 1
}
"""


from langchain_openai import ChatOpenAI
from langchain.messages import SystemMessage, HumanMessage
from enums.LLM import LLM

label0_result = ChatOpenAI(model=LLM.GPT4_1_MINI).invoke(
    [SystemMessage(content=system_prompt), HumanMessage(content=human_prompt_0)]
)
label0_result.content

label1_result = ChatOpenAI(model=LLM.GPT4_1_MINI).invoke(
    [SystemMessage(content=system_prompt), HumanMessage(content=human_prompt_0)]
)
label1_result.content


label0_samples = []
label1_samples = []


TARGET = 1000

while len(label0_samples) < TARGET:
    sample = ChatOpenAI(model=LLM.GPT4_1_MINI).invoke(
        [SystemMessage(content=system_prompt), HumanMessage(content=human_prompt_0)]
    )
    if sample:
        label0_samples.append(sample.content)


while len(label1_samples) < TARGET:
   sample = ChatOpenAI(model=LLM.GPT4_1_MINI).invoke([SystemMessage(content=system_prompt), HumanMessage(content=human_prompt_1)])   
   if sample:
        label1_samples.append(sample.content)

import random, json
all_samples = label0_samples + label1_samples
random.shuffle(all_samples)

with open("stt_binary_dataset.json", "w", encoding="utf-8") as f:
    json.dump(all_samples, f, ensure_ascii=False, indent=2)


In [90]:
import json
from pathlib import Path

def load_rows(path="stt_binary_dataset.json"):
    raw = json.loads(Path(path).read_text(encoding="utf-8"))
    rows = []
    for r in raw:
        if isinstance(r, str):
            try:
                r = json.loads(r)
            except:
                continue
        if not isinstance(r, dict): 
            continue
        if "stt_text" not in r or "label" not in r:
            continue
        t = str(r["stt_text"]).strip()
        if not t:
            continue
        y = int(r["label"])
        if y not in (0, 1):
            continue
        rows.append({"text": t, "label": y})
    return rows

rows = load_rows()
print("rows:", len(rows))
print(rows[:2])

rows: 1994
[{'text': '핑구 바람슛 甲乙丙合え', 'label': 1}, {'text': '보스 던전 어떻게 찾아', 'label': 0}]


In [94]:
import numpy as np
from datasets import Dataset
from sklearn.model_selection import train_test_split

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)

# ✅ 추천 인코더 (한국어/영어/다국어에 강함)
MODEL_NAME = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"

train_rows, valid_rows = train_test_split(rows, test_size=0.1, random_state=42, stratify=[r["label"] for r in rows])
ds_train = Dataset.from_list(train_rows)
ds_valid = Dataset.from_list(valid_rows)

tok = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)

def preprocess(batch):
    return tok(batch["text"], truncation=True, max_length=64)  # STT는 짧아서 64면 충분

ds_train = ds_train.map(preprocess, batched=True)
ds_valid = ds_valid.map(preprocess, batched=True)

collator = DataCollatorWithPadding(tok)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    acc = (preds == labels).mean()
    return {"accuracy": float(acc)}

args = TrainingArguments(
    output_dir="stt_miniLM_cls",
    learning_rate=2e-5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=128,
    num_train_epochs=3,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps=50,
    fp16=True,  # GPU 있으면
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=ds_train,
    eval_dataset=ds_valid,
    tokenizer=tok,
    data_collator=collator,
    compute_metrics=compute_metrics
)

trainer.train()
trainer.save_model("stt_miniLM_cls_final")
tok.save_pretrained("stt_miniLM_cls_final")
print("saved: stt_miniLM_cls_final")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map: 100%|██████████| 200/200 [00:00<00:00, 58294.70 examples/s]
/var/folders/fx/bdj1bhw11037xfwjzgrx6vc80000gn/T/ipykernel_16031/2362458146.py:49: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 2, 'bos_token_id': 0, 'pad_token_id': 1}.
/Users/seobi/PythonProjects/ProjectML/.venv/lib/python3.12/site-packages/torch/ut

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.301916,0.990000
2,0.415000,0.128509,1.000000
3,0.415000,0.101787,1.000000


/Users/seobi/PythonProjects/ProjectML/.venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/seobi/PythonProjects/ProjectML/.venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


saved: stt_miniLM_cls_final


In [98]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

MODEL_NAME = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"

tok = AutoTokenizer.from_pretrained(MODEL_NAME)  # ✅ 여기만 원본으로
model = AutoModelForSequenceClassification.from_pretrained("stt_miniLM_cls_final").eval()

@torch.no_grad()
def predict(text: str):
    x = tok(text, return_tensors="pt", truncation=True, max_length=64)
    logits = model(**x).logits
    probs = torch.softmax(logits, dim=-1)[0].cpu().numpy()
    return probs.argmax()

print(predict("이벤터 어또케"))
# print(predict("꽉!끼!떡!"))
# print(predict("꽉!끼!떡!"))

0


In [89]:
import joblib
model = joblib.load("stt_char_tfidf_svm_cal.joblib")

print("classes_:", model.classes_)  # 예: [0 1]

def p_label1(text: str) -> float:
    proba = model.predict_proba([text])[0]
    idx = list(model.classes_).index(1)  # 라벨 1 위치를 안전하게 찾기
    return float(proba[idx])

def predict_one(text: str, threshold=0.5):
    p1 = p_label1(text)
    pred = 1 if p1 >= threshold else 0
    return {"stt_text": text, "pred_label": pred, "p_label1": p1}

print(predict_one("이벤터 어또케"))           # 기대: pred_label=0
print(predict_one("핑구 바람슛 甲乙丙合え"))   # 기대: pred_label=1
print(predict_one("인벤 아이템 무슨키지"))     # 기대: pred_label=0
print(predict_one("적긔모"))                   # 기대: pred_label=1
print(predict_one("저 몬스터는 뭐야?"))

classes_: [0 1]
{'stt_text': '이벤터 어또케', 'pred_label': 0, 'p_label1': 0.0011737981821645304}
{'stt_text': '핑구 바람슛 甲乙丙合え', 'pred_label': 1, 'p_label1': 0.9963839720056229}
{'stt_text': '인벤 아이템 무슨키지', 'pred_label': 0, 'p_label1': 0.0016478660884164428}
{'stt_text': '적긔모', 'pred_label': 1, 'p_label1': 0.9839471982250018}
{'stt_text': '저 몬스터는 뭐야?', 'pred_label': 0, 'p_label1': 0.13074811606145645}
